# Simple Lead Shield for Neutron Radiation

This notebook is intended to create a simple test case of a lead shield for neutron radiation.

In [1]:
import openmc
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [3]:
lead = openmc.Material(name='lead');
lead.add_element('Pb',1)
lead.set_density=('g/cm3',11.34);

air = openmc.Material(name='air');
air.add_element('O',0.2);
air.add_element('N',0.8);
air.set_density('g/cm3',1.3e-3);

materials = openmc.Materials([air,lead]);

In [7]:
length = 50.;# cm
width = 10.; # cm
height = 10.; # cm
box = openmc.model.RectangularParallelepiped(0,length,0,width,
                                             0,height,boundary_type='vacuum');

# front wall of shield 5 cm from left boundary
fwall = openmc.XPlane(x0=5.);

shield_thickness = 5.; # cm
bwall = openmc.XPlane(x0=5.+shield_thickness);

# detector is a 3 cm radius sphere placed 15 cm from back wall of shield
detector = openmc.Sphere(x0=5.0+shield_thickness+15.,y0=0,z0=0,r=3.0)

c1 = openmc.Cell();
c1.region = -box & -fwall
c1.fill = air

c2 = openmc.Cell();
c2.region = -box & +fwall & -bwall
c2.fill = lead



c3 = openmc.Cell();
c3.region = -box & +bwall & +detector
c3.fill = air

det = openmc.Cell();
det.region = -detector
det.fill = None

root_univ = openmc.Universe();
root_univ.add_cells([c1,c2,c3,det]);

geometry = openmc.Geometry();
geometry.root_universe = root_univ;

materials.export_to_xml();
geometry.export_to_xml();

In [8]:
flux = openmc.Tally(name='flux');
energy,dose = openmc.data.dose_coefficients('neutron','AP');
dose_filter = openmc.EnergyFunctionFilter(energy,dose);
surface_filter = openmc.SurfaceFilter(detector);

flux.filters = [dose_filter,surface_filter];
flux.scores = ['current'];
tallies = openmc.Tallies([flux]);
tallies.export_to_xml();


In [9]:
point = openmc.stats.Point();
energy = openmc.stats.Watt(); # default is for U-235 fission from thermal neutron(?)
source = openmc.Source(space=point,energy=energy,particle='neutron');

settings = openmc.Settings();
settings.run_mode = 'fixed source';
settings.batches = 100;
settings.particles = 300000;
settings.generations_per_batch = 1;
settings.source = source;

settings.export_to_xml();
